In [76]:
# !usr/env python 3.5
from load_data import Download_Video
from pretrained_model import video_classification
from datetime import datetime
import os

In [2]:
# load data
video_num = 100
frame_count = 10
video_time = 10
d = Download_Video(video_num, frame_count, video_time)
tic = datetime.now()
d.download()
toc = datetime.now()
print('Download time {0}'.format(toc-tic))
tic = datetime.now()
d.preprocess()
toc = datetime.now()
print('Process time {0}'.format(toc-tic))

In [9]:
model = video_classification()
curr_path = os.getcwd()
frame_dir = curr_path + '/datasets/frames'
num_videos = 100
features = model.load_features()

In [3]:
# !usr/env python 3.5
from load_data import Download_Video
from datetime import datetime
import os
import logging

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# load data
video_num = 20
frame_count = 10
video_time = 10
d = Download_Video(video_num, frame_count, video_time)
tic = datetime.now()
d.download_organizer()
toc = datetime.now()
print('Download time {0}'.format(toc-tic))
tic = datetime.now()

ERROR: wi3PGpmtxH8: YouTube said: This video is no longer available because the YouTube account associated with this video has been terminated.
ERROR: 9NYdnRAGvO8: YouTube said: This video is no longer available because the YouTube account associated with this video has been terminated.
ERROR: 2WDnKNLxPKE: YouTube said: "Dragon Ball Z - Episode 94 ..."
This video is no longer available due to a copyright claim by TOEI ANIMATION CO.,LTD..
ERROR: AZZME-uNC0I: YouTube said: This video does not exist.
ERROR: JfSMzxjIjbY: YouTube said: "Full House Korean ..."
This video is no longer available due to a copyright claim by Arastoo Saadati.


Download time 0:01:26.892676


In [ ]:
video_num = 20
frame_count = 10
video_time = 10
d = Download_Video(video_num, frame_count, video_time)
tic = datetime.now()
d.preprocess_organizer()
toc = datetime.now()
print('Process time {0}'.format(toc-tic))


[MoviePy] Running:
>>> /Users/sunjiajun/Library/Application Support/imageio/ffmpeg/ffmpeg.osx -y -i /Users/sunjiajun/cs231n_project/datasets/videos/video10.mp4 -ss 33.33 -t 10.00 -vcodec copy -acodec copy /Users/sunjiajun/cs231n_project/datasets/processed/processed_video10.mp4
[MoviePy] Running:
>>> /Users/sunjiajun/Library/Application Support/imageio/ffmpeg/ffmpeg.osx -y -i /Users/sunjiajun/cs231n_project/datasets/videos/video14.mp4 -ss 81.69 -t 10.00 -vcodec copy -acodec copy /Users/sunjiajun/cs231n_project/datasets/processed/processed_video14.mp4
[MoviePy] Running:
>>> /Users/sunjiajun/Library/Application Support/imageio/ffmpeg/ffmpeg.osx -y -i /Users/sunjiajun/cs231n_project/datasets/videos/video22.mp4 -ss 20.05 -t 10.00 -vcodec copy -acodec copy /Users/sunjiajun/cs231n_project/datasets/processed/processed_video22.mp4
[MoviePy] Running:
>>> /Users/sunjiajun/Library/Application Support/imageio/ffmpeg/ffmpeg.osx -y -i /Users/sunjiajun/cs231n_project/datasets/videos/video28.mp4 -ss 5